In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
import shutil
import argparse
import time
import json
from datetime import datetime
from collections import defaultdict
from itertools import islice
import pickle
import copy
import numpy as np
import cv2
from easydict import EasyDict
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import autograd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.parallel import DistributedDataParallel
from tensorboardX import SummaryWriter

from mvn.models.triangulation import  VolumetricTriangulationNet
from mvn.models.volumetric_adain import VolumetricTemporalAdaINNet
from mvn.models.v2v import AdaIN, V2VModel, Basic3DBlock, Res3DBlock, Upsample3DBlock, Pool3DBlock, EncoderDecorder
from mvn.models import v2v
from mvn.models.loss import KeypointsMSELoss, KeypointsMSESmoothLoss, KeypointsMAELoss, KeypointsL2Loss, VolumetricCELoss
from mvn.models import pose_hrnet, pose_resnet
from mvn.utils import img, multiview, op, vis, misc, cfg
from mvn.datasets.human36m import Human36MTemporalDataset, Human36MMultiViewDataset
from mvn.datasets import utils as dataset_utils

from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose
from mvn.utils import img
from mvn.utils import volumetric
from mvn.utils import op
from mvn.utils import vis
from mvn.utils import misc
from mvn.utils import cfg
from mvn.datasets import utils as dataset_utils
from mvn.utils.img import image_batch_to_numpy, denormalize_image,to_numpy

from tqdm import tqdm_notebook

from torch.nn.utils.rnn import pack_padded_sequence
from pytorch_convolutional_rnn.convolutional_rnn import Conv3dLSTM, Conv3dPeepholeLSTM

from train import setup_human36m_dataloaders

device = torch.cuda.current_device()

def get_n(m):
    s = 0
    for param in m.parameters():
        s+=param.nelement()
    print (m.__class__, s//1024)    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
batch_size = 3
torch.cat((torch.zeros(batch_size), torch.ones(batch_size))).long()

tensor([0, 0, 0, 1, 1, 1])

In [24]:
lstm3d = Conv3dLSTM(in_channels=17, 
                    out_channels=17,
                    bidirectional=True,
                    batch_first=True,
                    num_layers=2,
                    kernel_size=3)
x = torch.randn(1, 8, 17, 32,32,32)
.
output, _ = lstm3d(x, None)

In [38]:
c1 = torch.randn(1, 32, 32, 32, 3)
c2 = torch.empty(1,8, 32, 32, 32, 3)

r = torch.broadcast_tensors(c1,c2)

In [50]:
8%7

1

In [48]:
i = 5
torch.all(r[0][:,i,...] == c1)

tensor(1, dtype=torch.uint8)

In [20]:
output = output[:,-1,...]

In [21]:
output.shape

torch.Size([1, 34, 32, 32, 32])

In [14]:
out[0].shape, out[1][0].shape, out[1][1].shape

(torch.Size([1, 8, 17, 32, 32, 32]),
 torch.Size([1, 8, 17, 32, 32, 32]),
 torch.Size([1, 8, 17, 32, 32, 32]))

# ST-LSTM/

In [ ]:
dt = 8
batch_size = 3

x = torch.randn(batch_size, dt, 128, 32,32,32)


